In [1]:
import json
import numpy as np

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

import random
from sklearn.model_selection import KFold
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import GridSearchCV

from sklearn.metrics import accuracy_score
import nltk

In [2]:
# import relevant LMs package

import transformers
from transformers import BertTokenizer
from torch.utils.data import TensorDataset, DataLoader, random_split
from transformers import BertForSequenceClassification, AdamW
from transformers import get_linear_schedule_with_warmup
from sklearn.metrics import f1_score, accuracy_score


In [3]:
# cheak to use gpu or cpu
if torch.cuda.is_available():
    print("Use gpu")
    device = torch.device("cuda")
else:
    print("Use cpu")
    device = torch.device("cpu")

Use gpu


## KG

In [4]:
# KGs

import re
import pandas as pd
import bs4
import requests
import spacy
from spacy import displacy
nlp = spacy.load('en_core_web_sm')

from spacy.matcher import Matcher 
from spacy.tokens import Span 

import networkx as nx

import matplotlib.pyplot as plt
from tqdm import tqdm

pd.set_option('display.max_colwidth', 200)
%matplotlib inline

In [32]:
import csv
# import wikipedia sentences
import os

cwd = os.getcwd()  # Get the current working directory (cwd)
files = os.listdir(cwd)  # Get all the files in that directory  
print("Files in %r: %s" % (cwd, files))

candidate_sentences = pd.read_csv("\\Users\\wz\\Desktop\\context-aware-embedding-master\\Resources\\code\\input\\wiki_sentences_v2.csv")
candidate_sentences.shape

Files in 'C:\\Users\\wz\\Desktop\\context-aware-embedding-master\\Resources\\code': ['.DS_Store', '.ipynb_checkpoints', 'accuracy.py', 'Fun online example.ipynb', 'input', 'Messing around.ipynb', 'Prototype 0.0.ipynb', 'Prototype0.1.ipynb', 'query_trl.ipynb', 'runs', 'test_trainer', 'trl', 'wandb', 'yago2', '__pycache__']


(4318, 1)

In [33]:
candidate_sentences['sentence'].sample(5)

1624    vocals were provided by trivedi, raftaar, ayushmann khurrana, arijit singh, abhijeet srivastava, aakansha sharma, shadab faridi and altamash faridi.
2295                                                                                                          the trailer was released on december 22, 2011.
3128                                                                                                                 majority of the fans considered his ra.
636                                                                                                                      ii was released on august 28, 2009.
3494                                                                                            educational and sports films are exempt from classification.
Name: sentence, dtype: object

In [34]:
doc = nlp("the drawdown process is governed by astm standard d823")

for tok in doc:
    print(tok.text, "...", tok.dep_)

the ... det
drawdown ... compound
process ... nsubjpass
is ... auxpass
governed ... ROOT
by ... agent
astm ... compound
standard ... pobj
d823 ... punct


In [35]:
def get_entities(sent):
## chunk 1
  ent1 = ""
  ent2 = ""

  prv_tok_dep = ""    # dependency tag of previous token in the sentence
  prv_tok_text = ""   # previous token in the sentence

  prefix = ""
  modifier = ""

  #############################################################
  
  for tok in nlp(sent):
    ## chunk 2
    # if token is a punctuation mark then move on to the next token
    if tok.dep_ != "punct":
      # check: token is a compound word or not
      if tok.dep_ == "compound":
        prefix = tok.text
        # if the previous word was also a 'compound' then add the current word to it
        if prv_tok_dep == "compound":
          prefix = prv_tok_text + " "+ tok.text
      
      # check: token is a modifier or not
      if tok.dep_.endswith("mod") == True:
        modifier = tok.text
        # if the previous word was also a 'compound' then add the current word to it
        if prv_tok_dep == "compound":
          modifier = prv_tok_text + " "+ tok.text
      
      ## chunk 3
      if tok.dep_.find("subj") == True:
        ent1 = modifier +" "+ prefix + " "+ tok.text
        prefix = ""
        modifier = ""
        prv_tok_dep = ""
        prv_tok_text = ""      

      ## chunk 4
      if tok.dep_.find("obj") == True:
        ent2 = modifier +" "+ prefix +" "+ tok.text
        
      ## chunk 5  
      # update variables
      prv_tok_dep = tok.dep_
      prv_tok_text = tok.text
  #############################################################

  return [ent1.strip(), ent2.strip()]

In [36]:
get_entities("the film had 200 patents")

['film', '200  patents']

In [37]:
entity_pairs = []

for i in tqdm(candidate_sentences["sentence"]):
    entity_pairs.append(get_entities(i))

100%|█████████████████████████████████████████████████████████████████████████████| 4318/4318 [00:13<00:00, 331.77it/s]


In [38]:
entity_pairs[10:20]

[['we', 'tests'],
 ['m', 'international sales rights'],
 ['musician robbie robertson', 'soundtrack'],
 ['it', 'original music tracks'],
 ['it', 'reviewed  franchise'],
 ['she', 'accidentally  mystique'],
 ['military  forces', 'arrest'],
 ['train', 'vuk'],
 ['kota eberhardt', 'telepath selene gallio'],
 ['singer', '-']]

In [39]:
def get_relation(sent):
    doc = nlp(sent)

    # Matcher class object 
    matcher = Matcher(nlp.vocab)

    #define the pattern 
    pattern = [{'DEP':'ROOT'}, 
                {'DEP':'prep','OP':"?"},
                {'DEP':'agent','OP':"?"},  
                {'POS':'ADJ','OP':"?"}] 

    matcher.add("matching_1",[pattern],on_match=None ) 

    matches = matcher(doc)
    k = len(matches) -1

    span = doc[matches[k][1]:matches[k][2]] 

    return(span.text)

In [40]:
get_relation("John completed the task")
len(candidate_sentences["sentence"])

4318

In [41]:
#  256 th  和 2700 th sentence 有问题 跳过这句 
# for i in tqdm(candidate_sentences['sentence'][257:2000]):
#     get_relation(i)
    

In [42]:
relations = [get_relation(i) for i in tqdm(candidate_sentences['sentence'][270:1700]) ]

100%|█████████████████████████████████████████████████████████████████████████████| 1430/1430 [00:04<00:00, 322.29it/s]


In [43]:
pd.Series(relations).value_counts()[:50]

is               120
was              109
released on       43
include           34
are               29
were              22
released          16
became            14
composed by       14
produced          12
has               11
have              11
become            11
made              10
considered         8
hired              8
need               7
included           7
introduced in      7
had                6
be                 6
been               6
starred in         6
produced by        5
's                 5
written by         5
scheduled          5
directed by        5
began              5
used               5
known as           5
released in        5
planned            5
george             4
given              4
called             4
hosted             4
gives              4
features           4
reported           4
cast as            4
designed           4
started in         4
start              4
rao                4
starred            4
began in           4
portrays     

In [44]:
# extract subject
source = [i[0] for i in entity_pairs][270:1700]

# extract object
target = [i[1] for i in entity_pairs][270:1700]

# print(len(source))
# print(len(target))
# print(len(relations))

kg_df = pd.DataFrame({'source':source, 'target':target, 'edge':relations})



In [45]:
kg_df

,source,target,edge
0,examples,unfriended friend request,include
1,one,noticeable side effects,is sleeplessness
2,early horror films,social controversy,created great
3,however horror movies,positive endings,present positive
4,satirical comedy drama,illicit love affair,is
...,...,...,...
1425,popular tune,audiences,believed
1426,bollywood,other products,used
1427,dual voting system,1956,developed in
1428,national film awards,also 1954,introduced in


In [54]:
def contains(source, target, relations):
    triple = np.array([source, target, relations])
    return (kg_df==triple).all(1).any()

contains("connie", "own",  "decides")

False

In [20]:
# create a directed-graph from a dataframe
G=nx.from_pandas_edgelist(kg_df, "source", "target", 
                          edge_attr=True, create_using=nx.MultiDiGraph())

In [21]:
plt.figure(figsize=(12,12))

pos = nx.spring_layout(G)
nx.draw(G, with_labels=True, node_color='skyblue', edge_cmap=plt.cm.Blues, pos = pos)
plt.show()

AttributeError: module 'scipy.sparse' has no attribute 'coo_array'

<Figure size 1200x1200 with 0 Axes>

## YAGO2

In [6]:
colnames=['subject', 'predicate', 'object'] 
YAGO = pd.read_csv("\\Users\\wz\\Desktop\\context-aware-embedding-master\\Resources\\code\\yago2\\yago2core_facts.clean.notypes.tsv" , names=colnames, header=None,sep ='\t')
# remove useless lines
# 1-260: xxx http://www.w3.org/2000/01/rdf-schema <yago>
YAGO = YAGO.iloc[261:557]
# remove the <> sign
YAGO['subject'] = YAGO['subject'].apply(lambda x: x[1:-1])
YAGO['predicate'] = YAGO['predicate'].apply(lambda x: x[1:-1])
YAGO['object'] = YAGO['object'].apply(lambda x: x[1:-1])
YAGO

,subject,predicate,object
261,Aristotle,influences,Ayn_Rand
262,Aristotle,influences,Alexander_the_Great
263,Aristotle,influences,Avicenna
264,Aristotle,influences,Albertus_Magnus
265,Aristotle,influences,Augustine_of_Hippo
...,...,...,...
552,Angola,participatedIn,Battle_of_Cuito_Cuanavale
553,Angola,participatedIn,South_African_Border_War
554,Angola,exports,wordnet_lumber_114943580
555,Angola,participatedIn,Battle_of_Kitombo


### Finetune against YAGO2

In [7]:
dataset = YAGO
# question answering format
dataset['query'] = dataset.apply(
    lambda row: row.subject + ' ' + row.predicate, axis=1)
# adjust format in list 
# dataset['query'] = [dataset['query'][i].split()[:2] for i in range(100)]
# dataset['query'] = dataset['query'].apply(lambda x: ' '.join(x))


In [8]:
dataset

,subject,predicate,object,query
261,Aristotle,influences,Ayn_Rand,Aristotle influences
262,Aristotle,influences,Alexander_the_Great,Aristotle influences
263,Aristotle,influences,Avicenna,Aristotle influences
264,Aristotle,influences,Albertus_Magnus,Aristotle influences
265,Aristotle,influences,Augustine_of_Hippo,Aristotle influences
...,...,...,...,...
552,Angola,participatedIn,Battle_of_Cuito_Cuanavale,Angola participatedIn
553,Angola,participatedIn,South_African_Border_War,Angola participatedIn
554,Angola,exports,wordnet_lumber_114943580,Angola exports
555,Angola,participatedIn,Battle_of_Kitombo,Angola participatedIn


In [9]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("bert-base-cased")


def tokenize_function(examples):
    return tokenizer(examples["query"], padding="max_length", truncation=True)

In [10]:
dataset["query"] = dataset["query"].apply(lambda x: tokenizer(x, padding="max_length", truncation=True))
dataset["object"] = dataset["object"].apply(lambda x: tokenizer(x, padding="max_length", truncation=True))
dataset

,subject,predicate,object,query
261,Aristotle,influences,"[input_ids, token_type_ids, attention_mask]","[input_ids, token_type_ids, attention_mask]"
262,Aristotle,influences,"[input_ids, token_type_ids, attention_mask]","[input_ids, token_type_ids, attention_mask]"
263,Aristotle,influences,"[input_ids, token_type_ids, attention_mask]","[input_ids, token_type_ids, attention_mask]"
264,Aristotle,influences,"[input_ids, token_type_ids, attention_mask]","[input_ids, token_type_ids, attention_mask]"
265,Aristotle,influences,"[input_ids, token_type_ids, attention_mask]","[input_ids, token_type_ids, attention_mask]"
...,...,...,...,...
552,Angola,participatedIn,"[input_ids, token_type_ids, attention_mask]","[input_ids, token_type_ids, attention_mask]"
553,Angola,participatedIn,"[input_ids, token_type_ids, attention_mask]","[input_ids, token_type_ids, attention_mask]"
554,Angola,exports,"[input_ids, token_type_ids, attention_mask]","[input_ids, token_type_ids, attention_mask]"
555,Angola,participatedIn,"[input_ids, token_type_ids, attention_mask]","[input_ids, token_type_ids, attention_mask]"


In [11]:
dataset_used = dataset.drop(["subject","predicate"],axis =1)
from sklearn.model_selection import train_test_split
small_train_dataset, small_eval_dataset = train_test_split(dataset_used, shuffle = True, test_size=0.2)

In [12]:
from transformers import AutoModelForQuestionAnswering

model = AutoModelForQuestionAnswering.from_pretrained("bert-base-uncased")

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForQuestionAnswering: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForQuestionAnswering were not initialized from the model checkpoint at bert-base-uncased a

In [13]:
from transformers import TrainingArguments

training_args = TrainingArguments(output_dir="test_trainer")

In [14]:
from datasets import load_metric

metric = load_metric("accuracy")

In [15]:
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

In [16]:
from transformers import TrainingArguments

training_args = TrainingArguments(output_dir="test_trainer", evaluation_strategy="epoch")

In [17]:

from transformers import Trainer

trainer = Trainer(model=model,args=training_args,train_dataset=small_train_dataset, 
                  eval_dataset=small_eval_dataset,compute_metrics=compute_metrics)

In [18]:
trainer.train()

wandb: Currently logged in as: wz_ryan. Use `wandb login --relogin` to force relogin


KeyError: 20

## LM

In [19]:
%load_ext autoreload
%autoreload 2

import torch
import wandb
import time
import os
from tqdm import tqdm
import numpy as np
import pandas as pd
from random import choices
import matplotlib.pyplot as plt
tqdm.pandas()

from datasets import load_dataset

from transformers import GPT2Tokenizer
from transformers import AutoModelForSequenceClassification, AutoTokenizer

from trl.gpt2 import GPT2HeadWithValueModel, respond_to_batch
from trl.ppo import PPOTrainer
from trl.core import build_bert_batch_from_txt

config = {
    "lm_name": "lvwerra/gpt2-imdb",
    "ref_lm_name": "lvwerra/gpt2-imdb",
    "cls_model_name": "lvwerra/distilbert-imdb",
    "tk_name": "gpt2",
    "steps": 320,
    "batch_size": 32,
    "forward_batch_size": 4,
    "ppo_epochs": 4,   
    "txt_in_len": 2,
    "txt_out_len": 1,
    "lr": 1.41e-5,
    "init_kl_coef":0.2,
    "target": 6,
    "horizon":10000,
    "gamma":1,
    "lam":0.95,
    "cliprange": .2,
    "cliprange_value":.2,
    "vf_coef":.1, 
    "seed": 1,
}

np.random.seed(config['seed'])


In [20]:
# trails
wandb.init(name='trails #0', project='Testing_largebatch', config=config)

### Load pre-trained GPT2 language models

In [21]:
gpt2_model = GPT2HeadWithValueModel.from_pretrained(config['lm_name'])
gpt2_model_ref = GPT2HeadWithValueModel.from_pretrained(config['ref_lm_name'])
gpt2_tokenizer = GPT2Tokenizer.from_pretrained(config['tk_name'])

Some weights of GPT2HeadWithValueModel were not initialized from the model checkpoint at lvwerra/gpt2-imdb and are newly initialized: ['transformer.h.0.attn.masked_bias', 'transformer.h.1.attn.masked_bias', 'transformer.h.2.attn.masked_bias', 'transformer.h.3.attn.masked_bias', 'transformer.h.4.attn.masked_bias', 'transformer.h.5.attn.masked_bias', 'transformer.h.6.attn.masked_bias', 'transformer.h.7.attn.masked_bias', 'transformer.h.8.attn.masked_bias', 'transformer.h.9.attn.masked_bias', 'transformer.h.10.attn.masked_bias', 'transformer.h.11.attn.masked_bias', 'v_head.summary.weight', 'v_head.summary.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of GPT2HeadWithValueModel were not initialized from the model checkpoint at lvwerra/gpt2-imdb and are newly initialized: ['transformer.h.0.attn.masked_bias', 'transformer.h.1.attn.masked_bias', 'transformer.h.2.attn.masked_bias', 'transformer.h.3.attn.masked_

In [22]:
_ = gpt2_model.to(device)
_ = gpt2_model_ref.to(device)

In [23]:
wandb.watch(gpt2_model, log='all')

[]

### Load BERT classifier

In [24]:
device = "cuda:0"

sentiment_model = AutoModelForSequenceClassification.from_pretrained(config["cls_model_name"])
sentiment_tokenizer = AutoTokenizer.from_pretrained(config["cls_model_name"])

# model = model.to(device)

In [25]:
text = 'this movie was really bad!!'
output = sentiment_model.forward(sentiment_tokenizer.encode(text, return_tensors="pt"))
output

SequenceClassifierOutput(loss=None, logits=tensor([[ 2.3350, -2.7266]], grad_fn=<AddmmBackward0>), hidden_states=None, attentions=None)

In [26]:
output[0][0, 1]

tensor(-2.7266, grad_fn=<SelectBackward0>)

### Next Token Prediction with GPT2

In [27]:
from transformers import AutoModelForCausalLM, AutoTokenizer


print("\nBegin next-word using HF GPT-2 demo ")

toker = AutoTokenizer.from_pretrained("gpt2")
model = AutoModelForCausalLM.from_pretrained("gpt2")


Begin next-word using HF GPT-2 demo 


In [28]:
seq = "connie own"
print("\nInput sequence: ")
print(seq)

inpts = toker(seq, return_tensors="pt")
print("\nTokenized input data structure: ")
print(inpts)

inpt_ids = inpts["input_ids"]  # just IDS, no attn mask
print("\nToken IDs and their words: ")
for id in inpt_ids[0]:
    word = toker.decode(id)
    print(id, word)

    
with torch.no_grad():
    logits = model(**inpts).logits[:, -1, :]

print("\nAll logits for next word: ")
print(logits)
print(logits.shape)

pred_id = torch.argmax(logits).item()
print("\nPredicted token ID of next word: ")
print(pred_id)

pred_word = toker.decode(pred_id)
print("\nPredicted next word for sequence: ")
print(pred_word)



Input sequence: 
connie own

Tokenized input data structure: 
{'input_ids': tensor([[ 1102, 11952,   898]]), 'attention_mask': tensor([[1, 1, 1]])}

Token IDs and their words: 
tensor(1102) con
tensor(11952) nie
tensor(898)  own

All logits for next word: 
tensor([[-86.0371, -81.7703, -89.6548,  ..., -93.6331, -92.7520, -87.4399]])
torch.Size([1, 50257])

Predicted token ID of next word: 
262

Predicted next word for sequence: 
 the


In [29]:
def GPT_pred(seq):
    inpts = toker(seq, return_tensors="pt")
    inpt_ids = inpts["input_ids"]  # just IDS, no attn mask
        

    with torch.no_grad():
        logits = model(**inpts).logits[:, -1, :]
    pred_id = torch.argmax(logits).item()
    pred_word = toker.decode(pred_id)
    return logits, pred_id, pred_word
    

In [30]:
def pos_logit_to_reward(logit, task):
    """
    Take the positive sentiment logit and scale it for the task.
        task [negative]: reward = -logit
        task [positive]: reward = logit
    """
    for i in range(len(logit)):
        if task[i]=='[false]':
            logit[i] = -logit[i]
        elif task[i]=='[true]':
            pass
        else:
            raise ValueError('task has to be in [0, 1, 2]!')
    return logit

In [46]:
df = kg_df[:1000]

# device = torch.device("cpu")
#model(img_test.unsqueeze(0).cuda()).detach().cpu().clone().numpy()

# modify query as object + relation
# e.g connie owns 
df['query'] = df.apply(
    lambda row: row.source + ' ' + row.edge, axis=1)
# adjust format in list 
df['query'] = [df['query'][i].split()[:2] for i in range(1000)]
df['query'] = df['query'].apply(lambda x: ' '.join(x))

# special token
df['tokens'] = df['query'].apply(lambda x: gpt2_tokenizer.encode(' '+str(x), return_tensors="pt").to(device))
df['tokens'] = df['tokens'].apply(lambda x: x[0])
df['tokens'] = df['tokens'].apply(lambda x: x[:2])

# dropping invalid length in token column
drop_row_index = []
for i in range(1000):

    if len(df['tokens'].loc[i]) < 2:
        #df['tokens'].loc[i] = df['tokens'].loc[0].cuda().detach().cpu().clone().numpy()
        drop_row_index.append(i)
print(drop_row_index)
df = df.drop(df.index[drop_row_index])
        

        
# df.loc[(len(df['tokens']) < 2), 'tokens'] = df['tokens'].loc[0]
#df['tokens'] = df['tokens'].apply(lambda x: df['tokens'].loc[0] if len(x) < 2)



ctrl_str = ['[false]', '[true]']
ctrl_tokens = dict((s, gpt2_tokenizer.encode(s, return_tensors="pt").squeeze().to(device)) for s in ctrl_str)

C:\Users\wz\AppData\Local\Temp\ipykernel_25688\1157424971.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['query'] = df.apply(
C:\Users\wz\AppData\Local\Temp\ipykernel_25688\1157424971.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['query'] = [df['query'][i].split()[:2] for i in range(1000)]
C:\Users\wz\AppData\Local\Temp\ipykernel_25688\1157424971.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = valu

[20, 22, 37, 128, 146, 173, 190, 208, 219, 225, 232, 258, 478, 488, 531, 542, 577, 673, 684, 703, 714, 743, 753, 754, 777, 797, 838, 868, 891, 953, 963]


C:\Users\wz\AppData\Local\Temp\ipykernel_25688\1157424971.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['tokens'] = df['tokens'].apply(lambda x: x[:2])


In [47]:
df

,source,target,edge,query,tokens
0,examples,unfriended friend request,include,examples include,"[tensor(6096, device='cuda:0'), tensor(2291, device='cuda:0')]"
1,one,noticeable side effects,is sleeplessness,one is,"[tensor(530, device='cuda:0'), tensor(318, device='cuda:0')]"
2,early horror films,social controversy,created great,early horror,"[tensor(1903, device='cuda:0'), tensor(9961, device='cuda:0')]"
3,however horror movies,positive endings,present positive,however horror,"[tensor(2158, device='cuda:0'), tensor(9961, device='cuda:0')]"
4,satirical comedy drama,illicit love affair,is,satirical comedy,"[tensor(40557, device='cuda:0'), tensor(10997, device='cuda:0')]"
...,...,...,...,...,...
995,it,rated superhero site,is,it is,"[tensor(340, device='cuda:0'), tensor(318, device='cuda:0')]"
996,jenkins,1918 era london,make,jenkins make,"[tensor(474, device='cuda:0'), tensor(268, device='cuda:0')]"
997,gas,war,intended,gas intended,"[tensor(3623, device='cuda:0'), tensor(5292, device='cuda:0')]"
998,david,popular mechanics,said,david said,"[tensor(21970, device='cuda:0'), tensor(531, device='cuda:0')]"


### PPO & Optimization

In [48]:
# device = torch.device("cuda")
# device = "cuda:0"
#input = input.to(device)
#         model = model.to(device)

a = [0,0,1,0]
a = [i + random.random() for i in a ]
a

[0.6394267984578837,
 0.025010755222666936,
 1.2750293183691193,
 0.22321073814882275]

In [49]:
### KG check to modify reward value 

# given a range i.e. i*fbs to (i+1)*fbs
# should output a tensor([ 0, 0,  1, 1], device='cuda:0')

# pred is the reponse in batch_response
# query is the query_list

def KG_check(lower, upper):
    tensor_array = []
    for i in range(lower,upper):
        # e.g. dark knight not
#         print('lower is ',lower )
#         print('upper is ',upper )
#         print("i is ", i)
#         print(len(query_list))
#         print(len(batch_response))

        # bugs here todo query_list[i].split() could be one element only
        source = query_list[i].split()[0]
        target = batch_response[i]
        if len(query_list[i].split()) <2:
            
            relation = 'anything'
        else:
            relation = query_list[i].split()[1]
        
#         print(i)
#         print("source is ",source)
#         print("target is ",target)
#         print("relation is ",relation)
        if contains(source, target, relation):
            tensor_array.append(1)
        else:
            tensor_array.append(0)
            
    # rand modular Todo: Fix this
    c = random.randint(0, 3)
    tensor_array[c]=1
    # print(tensor_array)
    
    # add variant 
    tensor_array = [i + random.random() for i in tensor_array ]
    return torch.cuda.LongTensor(tensor_array)

# KG_check(28,32)

In [50]:
config = {
    "lm_name": "lvwerra/gpt2-imdb",
    "ref_lm_name": "lvwerra/gpt2-imdb",
    "cls_model_name": "lvwerra/distilbert-imdb",
    "tk_name": "gpt2",
    "steps": 3200,
    "batch_size": 32,
    "forward_batch_size": 4,
    "ppo_epochs": 4,   
    "txt_in_len": 2,
    "txt_out_len": 1,
    "lr": 1.41e-5,
    "init_kl_coef":0.2,
    "target": 6,
    "horizon":10000,
    "gamma":1,
    "lam":0.95,
    "cliprange": .2,
    "cliprange_value":.2,
    "vf_coef":.1, 
    "seed": 1,
}

# where is lvwerra/distilbert-imdb?????

In [51]:
# debug
torch.cuda.empty_cache()
# df_batch

In [52]:
# debug

# df_results = pd.DataFrame(game_data)
# df_results
#query_list

In [55]:
# for testing

# batch_size: 32 -> df_batch
# forward_batch_size: 4  -> fbs

ppo_trainer = PPOTrainer(gpt2_model, gpt2_model_ref, **config)
fbs = config['forward_batch_size']

for epoch in tqdm(range(int(np.ceil(config["steps"]/config['batch_size'])))):
    torch.cuda.empty_cache()
    logs = dict()
    game_data = dict()
    timing = dict()
    t0 = time.time()
    
    #### get a batch from the dataset and annotate tasks
    df_batch = df.sample(config['batch_size'])
    task_list = choices(ctrl_str, k=config['batch_size'])
    task_tensors = torch.stack([ctrl_tokens[t] for t in task_list])
    query_list = df_batch['query'].tolist()
    game_data['query'] = [t+q for t,q in zip(task_list, query_list)]
    
    
    #print(task_tensors)
    query_tensors = torch.stack((df_batch['tokens'].tolist()))
    #print("before ",query_tensors)
    query_tensors = torch.cat((task_tensors, query_tensors), axis=1)

    
    #### get response from gpt2
    t = time.time()
    response_tensors_array = []
    for i in range(int(config['batch_size']/fbs)):
        #print(int(config['batch_size']/fbs))
        # 8 loops in total
        # 0-4, 5-8, 9-12...
        
        # this is troublesome 
        # error: probability tensor contains either `inf`, `nan` or element < 0
        #  next_token = torch.multinomial(probs, num_samples=1).squeeze(1) in gpt2.py function respond_to_batch
        
        # response  = respond_to_batch(gpt2_model, query_tensors[i*fbs:(i+1)*fbs],txt_len=config['txt_out_len'])
        # we want a response of size [i*fbs:(i+1)*fbs]*1
        response_tensors = []
        for i in range(i*fbs, (i+1)*fbs):
            seq = query_list[i]
            inpts = toker(seq, return_tensors="pt")
            inpt_ids = inpts["input_ids"]  # just IDS, no attn mask
            for id in inpt_ids[0]:
                word = toker.decode(id)

            with torch.no_grad():
                logits = model(**inpts).logits[:, -1, :]
            
            
            pred_id = torch.argmax(logits)
        
            response = pred_id
            
            # print(response.item())
            response_tensors.append(response.item())
        response_tensors = torch.cuda.LongTensor(response_tensors)
        response_tensors_array.append(response_tensors)

    # response_tensors.size = 32 * 1  
    response_tensors = torch.cat(response_tensors_array,axis = -1 )
    response_tensors  = torch.reshape(response_tensors, (-1, 1)) # force size of response_tensors.size = 32 * 1  rather that 32 length list
    # print(response_tensors)
    #print([gpt2_tokenizer.decode(response_tensors[i, :]) for i in range(config['batch_size'])])
    batch_response = [gpt2_tokenizer.decode(response_tensors[i, :]) for i in range(config['batch_size'])] # list of length 32
    game_data['response'] = batch_response
    timing['time/get_response'] = time.time()-t
    #print( game_data['response'])

    #### tokenize text for sentiment analysis
    t = time.time()
    texts = [q + r for q,r in zip(query_list, game_data['response'])]
    sentiment_inputs, attention_masks = build_bert_batch_from_txt(texts, sentiment_tokenizer, device)    
    timing['time/build_input_sentiment'] = time.time()-t
        
    #### get sentiment score
    t = time.time()
    pos_logits = []
    for i in range(int(config['batch_size']/fbs)):
        input = {'input_ids':      sentiment_inputs[i*fbs:(i+1)*fbs],
                  'attention_mask': attention_masks[i*fbs:(i+1)*fbs],
                  'labels':         None
                 }      
        #res = sentiment_model.forward(**input)[0][:, 1].detach()
        
        device = torch.device("cuda")
        sentiment_model = sentiment_model.to(device)
        res = sentiment_model.forward(sentiment_inputs[i*fbs:(i+1)*fbs],
                                      attention_masks[i*fbs:(i+1)*fbs])[0][:, 1].detach()
#         print(res)
#         print(res.size())

        # modify the rewards based on the KG
#         print('lower is ',i*fbs )
#         print('upper is ',(i+1)*fbs )
        res = KG_check(i*fbs, (i+1)*fbs)

        pos_logits.append(res)
    
    rewards = pos_logit_to_reward(torch.cat(pos_logits), task_list)
    timing['time/get_sentiment_preds'] = time.time()-t


    #### Run PPO training 
    t = time.time()
    stats = ppo_trainer.step(query_tensors, response_tensors, rewards)
    
    # todo fix this 
    # enforce nan to zero for wandb to run
    stats = np.nan_to_num(stats)
    #print(stats)
    timing['time/optimization'] = time.time()-t
     
    #### Log everything
    timing['time/epoch'] = time.time()-t0
    table_rows = [list(r) for r in zip(game_data['query'], game_data['response'], rewards.cpu().tolist())]
    logs.update({'game_log':wandb.Table(
        columns=['query', 'response', 'reward'],
        rows=table_rows)})
    logs.update(timing)
    # comment to remove the nan issues
    #logs.update(stats)
    
    # only support for float tensor
    rewards = rewards.type(torch.FloatTensor)
    
    logs['env/reward_mean'] = torch.mean((rewards)).cpu().numpy()
    logs['env/reward_std'] = torch.std(rewards).cpu().numpy()
    logs['env/reward_dist'] = rewards.cpu().numpy()
    for ctrl_s in ctrl_str:
        key = 'env/reward_'+ctrl_s.strip('[]')
        logs[key] = np.mean([r for r, t in zip(logs['env/reward_dist'], task_list) if t==ctrl_s])
#     WandbCallback(log=None)
    wandb.log(logs)

  0%|                                                                                          | 0/100 [00:00<?, ?it/s]C:\Users\wz\AppData\Local\Temp\ipykernel_25688\34421173.py:44: DeprecationWarning: an integer is required (got type float).  Implicit conversion to integers using __int__ is deprecated, and may be removed in a future version of Python.
  return torch.cuda.LongTensor(tensor_array)
100%|████████████████████████████████████████████████████████████████████████████████| 100/100 [10:34<00:00,  6.34s/it]


In [ ]:
# debug


#     cuda0 = torch.device('cuda:0')

#     for i in range(config['batch_size']):
#         current_row = df_batch['tokens'].iloc[i]
    
#         if type(current_row) is np.ndarray:
#             #print('before ', current_row[0])
#             #print(i)
#             val = [current_row[0], current_row[1]]
#             df_batch['tokens'].iloc[i] = list((torch.cuda.LongTensor([val[0]]),torch.cuda.LongTensor([val[1]])))
#             print(i)
#             print(df_batch['tokens'].iloc[i])

In [ ]:
# i = [6096, 9000]
# list((torch.cuda.LongTensor(i[0]),torch.cuda.LongTensor(i[1])))
# #torch.cpu.as_tensor(i)

### Example

In [ ]:
#### get a batch from the dataset
bs = 32
game_data = dict()
df_batch = df.sample(bs)
query_list = df_batch['query'].tolist()
game_data['query'] = query_list
for ctrl in ctrl_str:
    task_list = [ctrl] * bs
    task_tensors = torch.stack([ctrl_tokens[t] for t in task_list])

    query_tensors = torch.stack(df_batch['tokens'].tolist())
    query_tensors = torch.cat((task_tensors, query_tensors), axis=1)

    #### get response from gpt2 and gpt2_ref
    response_tensors  = respond_to_batch(gpt2_model, query_tensors, txt_len=config['txt_out_len'])
    game_data['response ' + ctrl] = [gpt2_tokenizer.decode(response_tensors[i, :]) for i in range(bs)]

    #### sentiment analysis of query/response pairs before/after
    texts = [q + r for q,r in zip(game_data['query'], game_data['response ' + ctrl])]
    sentiment_inputs, attention_masks = build_bert_batch_from_txt(texts, sentiment_tokenizer, device)    
    rewards = sentiment_model.forward(sentiment_inputs, attention_masks)[0][:, 1].detach()
    game_data['rewards ' + ctrl] = pos_logit_to_reward(rewards, task_list).cpu().numpy()

# store results in a dataframe
df_results = pd.DataFrame(game_data)
df_results

In [ ]:
# use BERT tokenizer

bert_tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)

# call tokenize.encode function to convert sentences

# # original sentence(X)
# print(' Original sentence: ', X[0])

# # tokenized sentence(X)
# print('Tokenized sentence: ', bert_tokenizer.tokenize(X[0]))

# # map sentence to token IDs
# print('Mapping to IDs: ', bert_tokenizer.convert_tokens_to_ids(bert_tokenizer.tokenize(X[0])))

# add [CLS] [SEP] [PAD] token as required 
# The encode funtion will 1. split into tokens 2. add [CLS] and [SEP] 3. map tokens to IDs
input_id = []
for s in X:
    encoded  = bert_tokenizer.encode(s,  # the sentence for encoding
                                     add_special_tokens = True,  # [CLS] and [SEP]
                                     max_length = 200, # maximum length of sentence
                                     pad_to_max_length = True, # padding length
                                     return_tensors = 'pt' # return type is tensor
                                    )
    input_id.append(encoded)


input_id = torch.cat(input_id, dim=0)
label_y = torch.tensor(Y) 

# set epoch and batch_size

epoch = 4
batch_size = 32

# split data for training and validation
data = TensorDataset(input_id,label_y)
data_len = len(data)
train, validation = random_split(data, [int(0.8*data_len), data_len - int(0.8*data_len)])

# create DataLoader for both training set and valiadation set
train_dl = DataLoader(train, batch_size = batch_size, shuffle = True)
val_dl = DataLoader(validation, batch_size = batch_size)



In [ ]:
# use BertForSequenceClassification as our model
model = BertForSequenceClassification.from_pretrained('bert-base-uncased', 
                                                      num_labels=4, 
                                                      output_attentions=False, # no attention weight
                                                      output_hidden_states=False # not need to return all hidden state )
                                                     )
# use cpu to train
model.cpu()

# optimizer and lr schedule

# convention is to use AdamW where W is weight decay fix from huggingface
optim = AdamW(model.parameters(), lr = 2e-5)
scheduler = get_linear_schedule_with_warmup(optim, 
                                           num_warmup_steps = 0, # default
                                           num_training_steps = len(train_dl)*epoch)

# define accuracy by accuracy_score

def accuracy(pred_label, y):
    pred_label = np.argmax(pred_label, axis =1).flatten()
    return accuracy_score(pred_label, y.flatten())



In [ ]:
# set the seed value to make sure the return are determistic

seed_value = 488
random.seed(seed_value)
np.random.seed(seed_value)
torch.manual_seed(seed_value)
torch.cuda.manual_seed_all(seed_value)
torch.backends.cudnn.deterministic = True

device = torch.device('cpu')


# training phase

for epc in range(epoch):
    # accumulated loss for training and validation
    training_loss = 0
    validation_loss = 0
    evaluation_acc = 0
    
    # training mode
    model.train()
    for step, batch in enumerate(train_dl):
        model.zero_grad()
        # each batch has 3 tensors: 1. input id 2. attention mask 3. label
        # e.g. batch[i].to(device)
        # follow the documentation on https://huggingface.co/transformers
        input_id = batch[0].to(device)
        att_mask = (batch[0]>0).to(device)
        input_label = batch[1].to(device)
        output = model(input_id, token_type_ids=None, attention_mask=att_mask, labels=input_label)
        loss = output[0]
        print("training batch ",step," with the loss of ", loss.item())
        training_loss += loss.item()
        # backprop
        loss.backward()
        # clip the gradient to 1.0, prevent exploding gradients
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
        # update parameter
        optim.step() 
        # update lr
        scheduler.step()
        
    # eval mode
    model.eval()
    for step, batch in enumerate(val_dl):
        # indicate the model not to compute gradients !
        with torch.no_grad():
            input_id = batch[0].to(device)
            att_mask = (batch[0]>0).to(device)
            input_label = batch[1].to(device)
            output = model(input_id, token_type_ids=None, attention_mask=att_mask, labels=input_label)
            loss = output[0] 
            print("val batch ",step," with the loss of ", loss.item())
            validation_loss += loss.item()
            pred = output[1].detach().cpu().numpy()
            ground_truth_label = batch[1].to('cpu').numpy()
            evaluation_acc += accuracy(pred, ground_truth_label)
            
    print('Validation loss: ', validation_loss / len(val_dl))
    print('Overall accuracy: ', evaluation_acc / len(val_dl))

print('Training finished')

In [ ]:
# testing
# predicate test label

# add [CLS] [SEP] [PAD] token as required
input_id_test = []
for s in Xt:
    encoded  = bert_tokenizer.encode(s, add_special_tokens = True, max_length = 200, 
                                pad_to_max_length = True, return_tensors = 'pt')
    input_id_test.append(encoded)
    
input_id_test = torch.cat(input_id_test, dim=0)
test_data = TensorDataset(input_id_test)
test_dl = DataLoader(test_data, batch_size=batch_size)

model.eval()

pred_label = []
for step, (batch,) in enumerate(test_dl):
    if step % 10 == 0:
        print("processed with batch ", step)
    with torch.no_grad():
        output = model(batch.to(device), token_type_ids=None, attention_mask=(batch>0).to(device))
        pred = output[0].detach().cpu().numpy()
        pred_label.append(pred)
print('Test predicting finished')
final_preds = np.concatenate(pred_label, axis=0)



In [ ]:
# convert probabilites into labels
Y_test_pred = final_preds

Y_pred = np.zeros((Y_test_pred.shape[0],1))

for i in range (Y_test_pred.shape[0]):
    array = Y_test_pred[i]
    
    max_val = np.max(array)
    
    index_max = np.where(array == max_val)
    Y_pred[i] = index_max[0][0]
    
    
print(Y_pred)

In [ ]:
# write out the csv file
# first column is the id, it is the index to the list of test examples
# second column is the predction as an integer
fout = open("out.csv", "w")
fout.write("Id,Y\n")
for i, line in enumerate(Y_pred): # Y_test_pred is in the same order as the test data
    fout.write("%d,%d\n" % (i, line))
fout.close()


In [ ]:
### REFERENCE

# https://www.kaggle.com/hassanamin/bert-pytorch-cola-classification
# https://www.cnblogs.com/tangjianwei/p/13334327.html